In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pickle
pd.options.mode.chained_assignment = None 

C:\Users\mrakeshr\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def wmae(dataset, real, predicted):
    weights = dataset.IsHoliday.apply(lambda x: 5 if x else 1)
    return np.round(np.sum(weights*abs(np.array(real)-np.array(predicted)))/(np.sum(weights)), 2)

In [3]:
#Returns the predictions of weekly sales on the data passed


def final_fun_1(X):
    X['Date']=pd.to_datetime(X.Date)  #Converting the string type to pandas date_time
    #Extracting year,month, day, day of the year, week of the year, is month start, is month end features.    
    X['Year']=pd.DatetimeIndex(X['Date']).year  
    X['Month']=pd.DatetimeIndex(X['Date']).month
    X['Day']=pd.DatetimeIndex(X['Date']).day
    X.Type = X.Type.apply(lambda x: 3 if x == 'A' else(2 if x == 'B' else 1))
    X.IsHoliday= X.IsHoliday.apply(lambda x: 0 if x== False else 1)
    X=X.fillna(0)
    X=X.drop(columns=['CPI','Unemployment','Fuel_Price','MarkDown1','MarkDown2','MarkDown3',
                                      'MarkDown4','MarkDown5','Date' ])
    model=pickle.load(open("final_model", 'rb')) #loading the saved model
    return model.predict(X)

In [4]:
#Returns WMAE of the data given

def final_fun_2(X,Y):
    
    X['Date']=pd.to_datetime(X.Date)  #Converting the string type to pandas date_time
    #Extracting year,month, day, day of the year, week of the year, is month start, is month end features.    
    X['Year']=pd.DatetimeIndex(X['Date']).year  
    X['Month']=pd.DatetimeIndex(X['Date']).month
    X['Day']=pd.DatetimeIndex(X['Date']).day
    X.Type = X.Type.apply(lambda x: 3 if x == 'A' else(2 if x == 'B' else 1))
    X.IsHoliday= X.IsHoliday.apply(lambda x: 0 if x== False else 1)
    X=X.fillna(0)
    X=X.drop(columns=['CPI','Unemployment','Fuel_Price','MarkDown1','MarkDown2','MarkDown3',
                                      'MarkDown4','MarkDown5','Weekly_Sales' ])
    X.drop(labels='Date', axis=1, inplace=True)
    model=pickle.load(open("final_model", 'rb'))
    return wmae(X, Y, model.predict(X))
    
    


In [5]:
#Reading the data
train=pd.read_csv("train.csv/train.csv")
features=pd.read_csv("features.csv/features.csv")
stores=pd.read_csv("stores.csv")
test=pd.read_csv("test.csv/test.csv")

train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

#Merging the 3 datasets given based on the store and Date
features_stores=features.merge(stores,how='inner', on=['Store'])
features_stores.Date=pd.to_datetime(features_stores.Date)
data=train.merge(features_stores,how='inner', on=['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

test_data = test.merge(features_stores, how='inner', on=['Store','Date','IsHoliday']).sort_values(by=['Store',
                                                                            'Dept',
                                                                            'Date']).reset_index(drop=True)

In [6]:
final_fun_2(data.loc[:1], data['Weekly_Sales'].iloc[:1]) # WMAE for single data point

14266.15

In [7]:
final_fun_2(data, data['Weekly_Sales']) # WMAE for all the data points

750.34

In [8]:
final_fun_1(test_data.iloc[:1]) # Predictions for a single data point

array([28081.72183333])

In [9]:
final_fun_1(test_data) #Predictions for all the data points

array([28081.72183333, 21324.22875   , 19414.35783333, ...,
         789.19816667,   749.41641667,   702.68841667])